# Python для анализа данных 

*Татьяна Рогович, НИУ ВШЭ*


## Регулярные выражения
#### (Задачи и Решения)

На основе статьи с [habr.com](https://habr.com/ru/post/349860/)

In [1]:
import re

### 1. Регистрационные знаки транспортных средств
В России применяются регистрационные знаки нескольких видов. 
Общего в них то, что они состоят из цифр и букв. Причём используются только 12 букв кириллицы, имеющие графические аналоги в латинском алфавите — А, В, Е, К, М, Н, О, Р, С, Т, У и Х.

У частных легковых автомобилях номера — это буква, три цифры, две буквы, затем две или три цифры с кодом региона. У такси — две буквы, три цифры, затем две или три цифры с кодом региона. Есть также и другие виды, но в этой задаче они не понадобятся.

Нам требуется определить, является ли последовательность букв корректным номером указанных двух типов, и если является, то каким.

На вход даются строки, которые претендуют на то, чтобы быть номером. На выходе нужно указать тип номера. Буквы в номерах — заглавные русские. Маленькие и английские для простоты можно игнорировать.

Примеры:
- С227НА777 - Private
- КУ22777 - Taxi
- Т22В7477 - Not Valid
- М227К19У9 - Not Valid
- *пробел*С227НА777 - Not Valid

In [14]:
def check(num):
    resP = re.match(r'[АВЕКМНОРСТУХ]\d{3}[АВЕКМНОРСТУХ]{2}\d{2,3}\b',num)
    if resP != None:
        print("Private")
        return
    resT = re.match(r'[АВЕКМНОРСТУХ]{2}\d{5,6}\b',num)
    if resT != None:
        print("Taxi")
    else:
        print("Not Valid")

In [13]:
number = "С227НА777"
check(number)

Private


In [8]:
number = "КУ22777"
check(number)

Taxi


In [15]:
number = "КУ2277710"
check(number)

Not Valid


In [5]:
number = "Т22В7477"
check(number)

Not Valid


In [6]:
number = "М227К19У9"
check(number)

Not Valid


In [7]:
number = " С227НА777"
check(number)

Not Valid


В этом задании создали два паттерна для определения частного номера и номера такси. Каждый соответствует своему типу номера:
для частного мы сначала ищем одну букву , потом последовательность из цифр длиной строго 3 символа `\d{3}`, снова две буквы и последовательность из 2 или 3 чисел для номера региона `\d{2,3}`.

Для такси сначала строго 2 буквы, и 5 или 6 цифр (3 цифры и потом номера региона от 2 до 3 чисел \d{2,3}).

В конце добавляем \b для обзначения конца номера (потому что без нее подстрока тоже проходила бы сравнение).

### 2. Количество слов
Слово — это последовательность из букв (русских или английских), внутри которой могут быть дефисы.
На вход даётся текст, посчитаем, сколько в нём слов.

**Пример:**

Он --- серо-буро-малиновая редиска!! \>>>:->  А не кот. www.kot.ru

**Вывод:** 9

In [16]:
re.findall(r'[a-zA-Zа-яА-Я]+(?:-[a-zA-Zа-яА-Я]+)*',"Он --- серо-буро-малиновая редиска!! >>>:-> А не кот. www.kot.ru")

['Он', 'серо-буро-малиновая', 'редиска', 'А', 'не', 'кот', 'www', 'kot', 'ru']

In [17]:
len(re.findall(r'[a-zA-Zа-яА-Я]+(?:-[a-zA-Zа-яА-Я]+)*',"Он --- серо-буро-малиновая редиска!! >>>:-> А не кот. www.kot.ru"))

9

Здесь мы составляем список возможных последовательностей, которые у нас составляют слово: внутри [], будем писать все что подойдет, а это и английские буквы a-z, большие в том числе A-Z, и также русские а-яА-Я. Далее мы говорим что хотим взять все такие последовательности идущие подряд. Таким образом мы захватим все слова, но серо-буро-малиновая разобъется на три разных слова. Поэтому далее мы пишем опциональный блок, длина которого может быть как нулевой, так и бесконечно большой. Повторять мы будем аналогичный паттерн с буквами, НО перед нем должен быть строго один знак дефиса.

### 3. Поиск почты

Допустимый формат e-mail адреса регулируется стандартом RFC 5322. 
Если говорить вкратце, то e-mail состоит из одного символа @ (at-символ или собака), текста до собаки (Local-part) и текста после собаки (Domain part). Вообще в адресе может быть всякий беспредел (вкратце можно прочитать о нём в википедии). Довольно странные штуки могут быть валидным адресом, например: 
- "very.(),:;<>[]\".VERY.\"very@\\ \"very\".unusual"@[IPv6:2001:db8::1] 
- "()<>[]:,;@\\\"!#$%&'-/=?^_`{}| ~.a"@(comment)exa-mple 

Но большинство почтовых сервисов такие адреса не допускают. И мы тоже не будем :)

Будем рассматривать только адреса, имя которых состоит из не более, чем 64 латинских букв, цифр и символов '._+-, а домен — из не более, чем 255 латинских букв, цифр и символов .-. Ни Local-part, ни Domain part не может начинаться или заканчиваться на .+-.

На вход даётся текст. Необходимо вывести все e-mail адреса, которые в нём встречаются. Стоит также помнить, что домен может быть двойным или даже тройным `wiki.hse.ru`

**Пример:**

Иван Иванович! Нужен ответ на письмо от ivanoff@ivan-chai.ru. Не забудьте поставить в копию serge'o-lupin@mail.ru- это важно.

**Вывод:**

ivanoff@ivan-chai.ru 

serge'o-lupin@mail.ru

In [18]:
st = "Иван Иванович! Нужен ответ на письмо от ivanoff@ivan-chai.ru. Не забудьте поставить в копию serge'o-lupin@mail.ru- это важно."
re.findall(r'\b[a-zA-z0-9-._]+@[a-zA-z0-9-._]{1,}\.[a-zA-Z]+', st)

['ivanoff@ivan-chai.ru', 'o-lupin@mail.ru']

Объяснение паттерна можно посмотреть в основном блокноте по регексу.

### 4. Замена времени
Вовочка подготовил одно очень важное письмо, но везде указал неправильное время. 
Поэтому нужно заменить все вхождения времени на строку (TBD). Время — это строка вида HH:MM:SS или HH:MM, в которой HH — число от 00 до 23, а MM и SS — число от 00 до 59.

**Пример**

Уважаемые! Если вы к 09:00 не вернёте чемодан, то уже в 09:00:01 я за себя не отвечаю. PS. С отношением 25:50 всё нормально!

**Вывод**

Уважаемые! Если вы к (TBD) не вернёте чемодан, то уже в (TBD) я за себя не отвечаю. PS. С отношением 25:50 всё нормально!

In [4]:
st = "Уважаемые! Если вы к 09:00 не вернёте чемодан, то уже в 09:00:01 я за себя не отвечаю. PS. С отношением 25:50 всё нормально!"
re.sub(r'(?:[01]\d|2[0-3]):(?:[0-5]\d)(?:[:0-5]+\d)*','(TBD)',st)

'Уважаемые! Если вы к (TBD) не вернёте чемодан, то уже в (TBD) я за себя не отвечаю. PS. С отношением 25:50 всё нормально!'

Не пугайтесь такого выражения, тут все довольно просто. Cкобки `(?:...)` позволяют локализовать часть шаблона, внутри которой происходит перечисление. Например, шаблон (?:он|тот) (?:шёл|плыл) соответствует каждой из строк «он шёл», «он плыл», «тот шёл», «тот плыл», и является синонимом он шёл|он плыл|тот шёл|тот плыл.

В случае нашего примера он соответствует: `[01]` - числа либо 0, либо 1, затем идет любая цифра `\d`. Таким образом мы покрыли все числа от 00 до 19. Но какже случай 20-23 часов? Это мы покрываем следующим условием, снова делая выбор между вариантами 00-19 `|` 20-23. `2` - нам нужна только цифра 2, а `[0-3]` говорит что подойдут числа от 0 до 3. Так мы разобрались с часами. Как можно видеть, последнее число 25 мы уже отсеяли.

In [5]:
re.sub(r'(?:[01]\d|2[0-3])','(TBD)',st)

'Уважаемые! Если вы к (TBD):(TBD) не вернёте чемодан, то уже в (TBD):(TBD):(TBD) я за себя не отвечаю. PS. С отношением 25:50 всё нормально!'

Как мы видим, наш шаблон заменил все числа на (TBD), в том числе минуты и секунды, потому что они прошли наш шаблон. Однако, время вида `23:49` уже будет заменено на `(TBD):49`. А `25` не прошли через шаблон для "часов".

Далее у нас всегда идет символ `:` и новый блок с минутами. Тут все почти как с часами: `[0-5]` любое число от 0 до 5, `\d`-  любая цифра.

In [7]:
re.sub(r'(?:[01]\d|2[0-3]):(?:[0-5]\d)','(TBD)',st)

'Уважаемые! Если вы к (TBD) не вернёте чемодан, то уже в (TBD):01 я за себя не отвечаю. PS. С отношением 25:50 всё нормально!'

Когда нет секунд, мы уже полностью корректно заменяем время на (TBD) и чтобы корректно заменять время с секундами, добавим такой же как для минут, опциональный блок. Почему он опциональный? Потому что его длина может быть `0`. `*` говорит нам, что слева от нее стоит паттерн, который может появляться от 0 и более раз. Сам же паттерн идентичен минутам `(?:[:0-5]+\d)`, кроме добавления в него `:`, потому двоеточие является частью секунд.

In [8]:
re.sub(r'(?:[01]\d|2[0-3]):(?:[0-5]\d)(?:[:0-5]+\d)*','(TBD)',st)

'Уважаемые! Если вы к (TBD) не вернёте чемодан, то уже в (TBD) я за себя не отвечаю. PS. С отношением 25:50 всё нормально!'

### 5. Аббревиатуры
Владимир устроился на работу в одно очень важное место. И в первом же документе он ничего не понял, 
там были сплошные ФГУП НИЦ ГИДГЕО, ФГОУ ЧШУ АПК и т.п. Тогда он решил собрать все аббревиатуры, чтобы потом найти их расшифровки на http://sokr.ru/. Помогите ему.

Будем считать аббревиатурой слова только лишь из заглавных букв (как минимум из двух). Если несколько таких слов разделены пробелами, то они 
считаются одной аббревиатурой.

**Пример**

Это курс информатики соответствует ФГОС и ПООП, это подтверждено ФГУ ФНЦ НИИСИ РАН

**Вывод**
- ФГОС 
- ПООП 
- ФГУ ФНЦ НИИСИ РАН

In [25]:
st = "Это курс информатики соответствует ФГОС и ПООП, это подтверждено ФГУ ФНЦ НИИСИ РАН"
re.findall(r'((?:[А-Я]{2,}\s){1,}[А-Я]{2,}|[А-Я]{2,})',st)

['ФГОС', 'ПООП', 'ФГУ ФНЦ НИИСИ РАН']

Давайте разбираться. Смысл тут похожий как в прошлой задаче. Сначала мы ищем вариант с несколькими идущими подряд аббревиатурами, а потом одиночный. Используемо опять же `(|)` т.е. `ИЛИ`. Начнем с поиска заглавных букв

In [26]:
re.findall(r'(?:[А-Я]{2,})',st)

['ФГОС', 'ПООП', 'ФГУ', 'ФНЦ', 'НИИСИ', 'РАН']

Так мы найдем все последовательности заглавных букв `[А-Я]` длиной от 2 и больше `{2,}`

In [22]:
re.findall(r'(?:[А-Я]{2,}\s){1,}',st)

['ФГОС ', 'ФГУ ФНЦ НИИСИ ']

Теперь мы нашли все прошлые последовательности + один пробел `\s`. А также указали, что мы ищем одну и более таких последовательностей с пробелом `(){1,}`. Получили одну одиночную аббревиатуру `ФГОС ` и аббревиатуру состояющую из частей `ФГУ ФНЦ НИИСИ `, без последений части. Все потому что после нее нет пробела.

In [23]:
re.findall(r'(?:[А-Я]{2,}\s){1,}[А-Я]{2,}',st)

['ФГУ ФНЦ НИИСИ РАН']

Тут мы указываем что закончить мы должны тоже аббревиатурой и притом только одной и без пробела: `[А-Я]{2,}`. Получили только составную аббревиатуры. Теперь же добавим выбор `(|)` и вернем одиночные аббревиатуры.

In [24]:
re.findall(r'((?:[А-Я]{2,}\s){1,}[А-Я]{2,}|[А-Я]{2,})',st)

['ФГОС', 'ПООП', 'ФГУ ФНЦ НИИСИ РАН']

### 6. Шифровка
Владимиру потребовалось срочно запутать финансовую документацию. Но так, чтобы это было обратимо. 
Он не придумал ничего лучше, чем заменить каждое целое число (последовательность цифр) на его куб. Нужно ему помочь.

**Пример**

Было закуплено 12 единиц техники по 410.37 рублей.

**Вывод**

Было закуплено 1728 единиц техники по 68921000.50653 рублей.

In [27]:
def repl(m):
    mi = int(m[0])
    return str(mi*mi*mi) 

st = "Было закуплено 12 единиц техники по 410.37 рублей."
re.sub(r'\d+', repl, st)

'Было закуплено 1728 единиц техники по 68921000.50653 рублей.'

Как мы видим, функции `sub` можно передать функцию, в которую мы передадим найденный по выражению объект. Далее с ним можно сделать все что угодно (в нашем случае возвели в куб) и потом вставить обратно. Главное не забыть перевести обратно в строковый тип. Ну а выражение простое: ищем все цифровые последовательности любой длины.

### 7. То ли акростих, то ли акроним, то ли апроним
Акростих — осмысленный текст, сложенный из начальных букв каждой строки стихотворения.
Акроним — вид аббревиатуры, образованной начальными звуками (напр. НАТО, вуз, НАСА, ТАСС), которое можно произнести слитно (в отличие от аббревиатуры, которую произносят «по буквам», например: КГБ — «ка-гэ-бэ»). 
На вход даётся текст. Выведем слитно первые буквы каждого слова. Буквы необходимо выводить заглавными. 

**Пример**

Московский государственный институт международных отношений

**Вывод**

МГИМО

In [28]:
st = "Московский государственный институт международных отношений"
acr = re.findall(r'\b[а-яА-Я]',st)
a = ''.join(acr).upper()
a

'МГИМО'

Или решение одной строкой

In [29]:
st = "микоян авиацию снабдил алкоголем, народ доволен работой авиаконструктора"
''.join(re.findall(r'\b[а-яА-Я]',st)).upper()

'МАСАНДРА'

Тут довольно простое решение, сначала мы ищем в начале слова `\b` буквы `[а-яА-Я]`. Получившиеся буквы из списка мы переводим в строку с применением метода `upper()`, который сделает их заглавными. Регулярные выражения очень мощная вещь при использовании с другими функциями. Также не стоит забывать, что некоторые вещи можно проще решить другими способами.

### 8.Форматирование номера телефона
Если вы когда-нибудь пытались собирать номера мобильных телефонов, то наверняка знаете, что почти любые 10 человек используют как минимум пяток различных способов записать номер телефона. Кто-то начинает с +7, кто-то просто с 7 или 8, а некоторые вообще не пишут префикс. Трёхзначный код кто-то отделяет пробелами, кто-то при помощи дефиса, кто-то скобками (и после скобки ещё пробел некоторые добавляют). После следующих трёх цифр кто-то ставит пробел, кто-то дефис, кто-то ничего не ставит. И после следующих двух цифр — тоже. В общем очень неудобно!


На вход даётся номер телефона, как его мог бы ввести человек. Необходимо его переформатировать в формат +7 123 456-78-90. Если с номером что-то не так, то нужно вывести строчку Fail!.

**Пример**

+7 123 456-78-90

8(123)456-78-90

7(123) 456-78-90

1234567890

123456789

+9 123 456-78-90

+7 123 456+78=90

+7(123 45678-90

**Вывод**

+7 123 456-78-90

+7 123 456-78-90

+7 123 456-78-90

+7 123 456-78-90

Fail!

Fail!

+7 123 456-78-90

+7 123 456-78-90

In [2]:
r = re.compile(r"(\+\d)(\d{3})(\d{3})(\d{2})(\d{2})")

def check_num(num):
    clean_num = re.sub(r'[\W]','',num)
    clean_num = re.sub(r'(^7|^8)','+7',clean_num)
    s = re.match(r'\+7\d{10}\b|\d{10}\b',clean_num)

    if s!= None:
        if len(s[0]) == 12:
            print(re.sub(r,r"\1 \2-\3-\4-\5",s[0]))
        elif len(s[0]) == 10 and len(num) == 10:
            st = "+7" + clean_num
            print(re.sub(r,r"\1 \2-\3-\4-\5",st))
        else:
            print("Fail!")
    else:
        print("Fail!")

In [3]:
num = "+7 123 456-78-90"
check_num(num)

num = "8(123)456-78-90"
check_num(num)

num = "7(123) 456-78-90"
check_num(num)

num = "1234567890"
check_num(num)

num = "123456789"
check_num(num)

num = "+9 123 456-78-90"
check_num(num)

num = "+7 123 456+78=90"
check_num(num)

num = "+7(123 45678-90"
check_num(num)

+7 123-456-78-90
+7 123-456-78-90
+7 123-456-78-90
+7 123-456-78-90
Fail!
Fail!
+7 123-456-78-90
+7 123-456-78-90


Обработку строк мы делает в функции и начинаем с `очищения` строки. `[\W]` заменим в ней все, что не число и не буква, склеив все числа в один (возможно) номер.

In [35]:
re.sub(r'[\W]','',num)

'71234567890'

In [36]:
clean_num = re.sub(r'[\W]','',num)

Теперь разберемся с проблемой `+7`,`7` и `8`. Найдем все `7` и `8` и заменим их на `+7`. Поиск совершаем только в начале строки `^`

In [37]:
clean_num = re.sub(r'(^7|^8)','+7',clean_num)
clean_num

'+71234567890'

Имеем очищенную строку. Теперь можно в ней искать наш паттерн номера телефона:`\+7` - ищем +7 в начале телефона, далее идет группа чисел длиной от 2 до 3 символов `(?:\d{2,3})`, которая повторяется ровно 4 раза `{4}`. В итоге получаем номер.
Итак, получили номер, но только если он был написан с `+7`,`7` или `8`. Проверяем на наличие найденой последовательности и если она есть, выводим ответ

In [38]:
s = re.match(r'\+7\d{10}\b|\d{10}\b',clean_num)
if s != None:
     if len(s[0]) == 12:
            print(re.sub(r,r"\1 \2-\3-\4-\5",s[0]))

+7 123-456-78-90


Ответ будет формировать путем разделения номера на группы: `r = re.compile(r"(\+\d)(\d{3})(\d{3})(\d{2})(\d{2})")`, каждая `()` отвечает за определенную группу, например `(\+\d)` найдет нам +7, это будет `первая группа`, 3 следующие цифры `(\d{3})` - `вторая группа` и т.д. И применив замену, выводим группы `\1`, `\2`, `\3`... в нужном нам формате, с пробелами и дефисами: `\1 \2-\3-\4-\5`

Далее рассмотрим случай когда указаны только 10 цифр номера. Проверяем длину строки и если все правильно, дописываем `+7` в начале и снова вставляем в строку нужные элементы срезом

In [36]:
if len(s[0]) == 10 and len(num) == 10:
    st = "+7"+clean_num
    print(s[0][:2]+" "+s[0][2:5]+" "+s[0][5:8]+"-"+s[0][8:10]+"-"+s[0][10:])

Если обе проверки правилились, то пишем что номера тут нет. Данный код можно улучшить и собирать номера из остальных случаев

### 9. Форматирование больших чисел
Большие целые числа удобно читать, когда цифры в них разделены на тройки запятыми. 
Переформатируйте целые числа в тексте

**Пример**

12 мало 

лучше 123 

1234 почти 

12354 хорошо 

стало 123456 

супер 1234567

**Вывод**

12 мало 

лучше 123 

1,234 почти 

12,354 хорошо 

стало 123,456 

супер 1,234,567

In [40]:
def repl(m):
    return '{:,}'.format(int(m[0]))

st = "12 мало лучше 123 1234 почти 12354 хорошо стало 123456 супер 1234567"
re.sub(r'\d+', repl, st)

'12 мало лучше 123 1,234 почти 12,354 хорошо стало 123,456 супер 1,234,567'

Снова воспользуемся функцией для замены чисел. `\d+` - получаем последовательность из чисел. Далее переводим ее в число и применяем встроенную в python функцию форматирования `format` с указанием того что хотим отображать знак разделения `{:,}`.